# Quantum Entanglement: Spooky Action at a Distance

## Introduction

**What is quantum entanglement?**  
Entanglement is a uniquely quantum phenomenon where two or more particles become correlated in such a way that measuring one particle **instantly** affects the state of the other, regardless of the distance between them. Einstein famously called this "spooky action at a distance."

**Why entanglement matters for quantum computing**  
Entanglement is a key resource for:
- **Quantum algorithms**: Many quantum algorithms (Shor's, Grover's) rely on entangled states
- **Quantum communication**: Entanglement enables quantum teleportation and secure communication
- **Quantum error correction**: Entangled states protect quantum information from noise
- **Quantum advantage**: Entanglement provides correlations impossible in classical physics

**Classical vs quantum correlations**  
Classical correlations can be explained by shared information (e.g., two envelopes with matching coins). Quantum correlations violate Bell inequalities, proving they cannot be explained by any classical "hidden variable" theory.

**What we'll demonstrate**  
In this notebook, we'll:
1. Compare classical vs quantum correlations
2. Create and visualize all four Bell states
3. Measure correlation matrices and entanglement witnesses
4. Demonstrate CHSH inequality violation
5. (Optional) Run on real quantum hardware

**Key insight**  
Classical correlations: $C \leq 2$  
Quantum correlations: $C \leq 2\sqrt{2} \approx 2.828$ (Bell's theorem violation!)

## Setup: Imports and Configuration

Let's import our tools and configure beautiful visualizations.

In [ ]:
# Quantum computing framework
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Statevector, partial_trace, entanglement_of_formation
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_bloch_multivector, plot_state_qsphere

# Numerical and visualization libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Custom utilities for beautiful plots
import sys
sys.path.append('..')
from utils.plotting import (
    configure_beautiful_plots,
    plot_histogram_comparison,
    plot_correlation_matrix,
    plot_multiple_histograms,
    COLORS
)

# Configure beautiful plotting style
configure_beautiful_plots()

# Set random seed for reproducibility
np.random.seed(42)

print("✅ All imports successful")
print("✅ Ready to explore quantum entanglement")

## Classical Correlations: Shared Coins

**What we'll do**  
Simulate classical correlations using two coins placed in envelopes. Alice and Bob each receive an envelope with matching coins (both heads or both tails).

**Why**  
This establishes the baseline for classical correlations. Even with perfect correlation (when they measure in the same basis), classical correlations are limited.

**How**  
Monte Carlo simulation: randomly prepare matching pairs, measure, compute correlation.

**Expected result**  
Perfect correlation when measuring in the same basis, but limited correlation overall.

In [ ]:
# Simulate classical correlated coins
n_trials = 10000

# Prepare random matching pairs
classical_pairs = np.random.randint(0, 2, size=n_trials)  # 0=both heads, 1=both tails

# Alice and Bob measure (0=heads, 1=tails)
alice_results = classical_pairs.copy()
bob_results = classical_pairs.copy()

# Compute correlation: +1 if same, -1 if different
classical_correlation = np.mean(2 * (alice_results == bob_results) - 1)

# Count outcomes
same_outcome = np.sum(alice_results == bob_results)
diff_outcome = n_trials - same_outcome

# Beautiful visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6), dpi=150)

# Outcome distribution
outcomes = ['Both Heads\n(00)', 'Both Tails\n(11)']
counts = [np.sum(classical_pairs == 0), np.sum(classical_pairs == 1)]
colors_bar = [COLORS['primary'], COLORS['secondary']]

bars = ax1.bar(outcomes, counts, color=colors_bar, alpha=0.8, edgecolor='black', linewidth=2)
ax1.set_ylabel('Frequency', fontsize=12, fontweight='bold')
ax1.set_title('Classical Correlated Pairs Distribution', fontsize=14, fontweight='bold')
ax1.grid(axis='y', alpha=0.3)

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(height)}',
            ha='center', va='bottom', fontsize=11, fontweight='bold')

# Correlation coefficient
corr_data = ['Agreement', 'Disagreement']
corr_counts = [same_outcome, diff_outcome]
colors_corr = [COLORS['success'], COLORS['classical']]

bars2 = ax2.bar(corr_data, corr_counts, color=colors_corr, alpha=0.8, edgecolor='black', linewidth=2)
ax2.set_ylabel('Frequency', fontsize=12, fontweight='bold')
ax2.set_title('Alice-Bob Agreement', fontsize=14, fontweight='bold')
ax2.grid(axis='y', alpha=0.3)

# Add value labels
for bar in bars2:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(height)}',
            ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.suptitle('Classical Correlations: Shared Information Model', fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

print("\n" + "="*70)
print("CLASSICAL CORRELATION RESULTS")
print("="*70)
print(f"Total trials: {n_trials}")
print(f"Perfect agreement: {same_outcome} ({100*same_outcome/n_trials:.1f}%)")
print(f"Correlation coefficient: {classical_correlation:.3f}")
print("\nInterpretation: Perfect classical correlation (C = 1.0)")
print("This can be explained by shared information in the envelopes.")
print("="*70)

## Creating Quantum Entanglement: Bell States

**What we'll do**  
Create all four maximally entangled Bell states using quantum circuits.

**The four Bell states:**
- $|\Phi^+\rangle = \frac{1}{\sqrt{2}}(|00\rangle + |11\rangle)$ — Fully correlated
- $|\Phi^-\rangle = \frac{1}{\sqrt{2}}(|00\rangle - |11\rangle)$ — Correlated with phase
- $|\Psi^+\rangle = \frac{1}{\sqrt{2}}(|01\rangle + |10\rangle)$ — Anti-correlated
- $|\Psi^-\rangle = \frac{1}{\sqrt{2}}(|01\rangle - |10\rangle)$ — Anti-correlated with phase

**How**  
Use Hadamard + CNOT gates (with optional X and Z gates for other Bell states).

**Expected result**  
Four distinct maximally entangled states with perfect correlations.

In [ ]:
def create_bell_state(state_type='phi_plus'):
    """
    Create one of the four Bell states.
    
    Parameters:
    -----------
    state_type : str
        One of 'phi_plus', 'phi_minus', 'psi_plus', 'psi_minus'
    
    Returns:
    --------
    QuantumCircuit with the specified Bell state
    """
    qc = QuantumCircuit(2)
    
    # All Bell states start with H-CNOT
    qc.h(0)
    qc.cx(0, 1)
    
    # Add gates to get other Bell states
    if state_type == 'phi_minus':
        qc.z(0)  # Add phase
    elif state_type == 'psi_plus':
        qc.x(0)  # Flip first qubit
    elif state_type == 'psi_minus':
        qc.x(0)  # Flip first qubit
        qc.z(0)  # Add phase
    
    return qc

# Create all four Bell states
bell_states = {
    'Φ⁺': create_bell_state('phi_plus'),
    'Φ⁻': create_bell_state('phi_minus'),
    'Ψ⁺': create_bell_state('psi_plus'),
    'Ψ⁻': create_bell_state('psi_minus')
}

# Get statevectors
statevectors = {name: Statevector(qc) for name, qc in bell_states.items()}

# Print all Bell states
print("="*70)
print("THE FOUR BELL STATES")
print("="*70)
for name, state in statevectors.items():
    print(f"\n{name}: {state.data}")
    print(f"   Probabilities: |00⟩={abs(state.data[0])**2:.3f}, |01⟩={abs(state.data[1])**2:.3f}, "
          f"|10⟩={abs(state.data[2])**2:.3f}, |11⟩={abs(state.data[3])**2:.3f}")
print("\n" + "="*70)

# Visualize one Bell state circuit
print("\nExample: Creating |Φ⁺⟩ = (|00⟩ + |11⟩)/√2")
display(bell_states['Φ⁺'].draw('mpl', style='iqp'))

## Measuring Bell States: Perfect Correlations

**What we'll do**  
Measure each Bell state multiple times and observe the correlation patterns.

**Why**  
Bell states exhibit perfect correlations: if Alice measures |0⟩ in the Φ⁺ state, Bob **always** measures |0⟩ too.

**Expected**  
- Φ⁺: Only |00⟩ and |11⟩ outcomes (correlated)
- Φ⁻: Only |00⟩ and |11⟩ outcomes (correlated with phase)
- Ψ⁺: Only |01⟩ and |10⟩ outcomes (anti-correlated)
- Ψ⁻: Only |01⟩ and |10⟩ outcomes (anti-correlated with phase)

In [ ]:
# Measure all Bell states
simulator = AerSimulator()
shots = 1000

bell_measurements = {}

for name, qc in bell_states.items():
    # Add measurements
    qc_measure = qc.copy()
    qc_measure.measure_all()
    
    # Run simulation
    job = simulator.run(qc_measure, shots=shots)
    counts = job.result().get_counts()
    bell_measurements[name] = counts

# Beautiful visualization of all four Bell state measurements
titles = [f"{name} State\n{theory}" for name, theory in [
    ('Φ⁺', '(|00⟩+|11⟩)/√2'),
    ('Φ⁻', '(|00⟩-|11⟩)/√2'),
    ('Ψ⁺', '(|01⟩+|10⟩)/√2'),
    ('Ψ⁻', '(|01⟩-|10⟩)/√2')
]]

data_list = [bell_measurements[name] for name in ['Φ⁺', 'Φ⁻', 'Ψ⁺', 'Ψ⁻']]

fig = plot_multiple_histograms(
    data_list,
    titles,
    overall_title='Four Bell States: Maximally Entangled Quantum Correlations',
    ncols=2
)
plt.show()

print("\n" + "="*70)
print("BELL STATE MEASUREMENT RESULTS")
print("="*70)
for name, counts in bell_measurements.items():
    print(f"\n{name}: {counts}")
    
    # Calculate correlation
    same = counts.get('00', 0) + counts.get('11', 0)
    diff = counts.get('01', 0) + counts.get('10', 0)
    
    if name in ['Φ⁺', 'Φ⁻']:
        print(f"   → Correlated (00 and 11): {same/shots*100:.1f}%")
    else:
        print(f"   → Anti-correlated (01 and 10): {diff/shots*100:.1f}%")

print("\n" + "="*70)
print("\n🎯 Key observation: No |01⟩ or |10⟩ in Φ states, no |00⟩ or |11⟩ in Ψ states!")
print("   This perfect correlation is a signature of quantum entanglement.")
print("="*70)

## Correlation Matrix Visualization

**What we'll do**  
Compute and visualize correlation matrices for Bell states.

**Why**  
Correlation matrices reveal the strength and pattern of correlations between qubits.

**How**  
For each Bell state, compute $\langle Z_i Z_j \rangle$ correlations and visualize as a heatmap.

**Expected result**  
Strong correlations (+1 or -1) between qubits in Bell states.

In [ ]:
# Compute correlation matrices for Bell states
def compute_correlation_from_counts(counts, n_qubits=2):
    """
    Compute ZZ correlation from measurement counts.
    
    Returns correlation coefficient: +1 (correlated), -1 (anti-correlated), 0 (uncorrelated)
    """
    total = sum(counts.values())
    correlation = 0
    
    for bitstring, count in counts.items():
        # Convert bitstring to list of bits
        bits = [int(b) for b in bitstring]
        
        # Z measurement: |0⟩ → +1, |1⟩ → -1
        z0 = 1 - 2 * bits[0]
        z1 = 1 - 2 * bits[1]
        
        # ZZ correlation
        correlation += (z0 * z1) * count / total
    
    return correlation

# Compute correlations for all Bell states
correlations = {}
for name, counts in bell_measurements.items():
    corr = compute_correlation_from_counts(counts)
    correlations[name] = corr

# Beautiful correlation visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 12), dpi=150)
axes = axes.flatten()

for idx, (name, counts) in enumerate(bell_measurements.items()):
    ax = axes[idx]
    
    # Create 2x2 correlation matrix (both qubits vs both qubits)
    corr_matrix = np.array([[1.0, correlations[name]], 
                            [correlations[name], 1.0]])
    
    # Plot heatmap
    im = ax.imshow(corr_matrix, cmap='RdBu', vmin=-1, vmax=1, aspect='auto')
    
    # Add text annotations
    for i in range(2):
        for j in range(2):
            text = ax.text(j, i, f'{corr_matrix[i, j]:.2f}',
                          ha="center", va="center", color="black", 
                          fontsize=14, fontweight='bold')
    
    ax.set_xticks([0, 1])
    ax.set_yticks([0, 1])
    ax.set_xticklabels(['Qubit 0', 'Qubit 1'], fontsize=11)
    ax.set_yticklabels(['Qubit 0', 'Qubit 1'], fontsize=11)
    ax.set_title(f'{name} Correlation Matrix', fontsize=13, fontweight='bold')
    
    # Add colorbar
    cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    cbar.set_label('⟨ZZ⟩ Correlation', fontsize=10)

plt.suptitle('Bell State Correlation Matrices', fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

print("\n" + "="*70)
print("CORRELATION COEFFICIENTS ⟨Z₀Z₁⟩")
print("="*70)
for name, corr in correlations.items():
    print(f"{name}: {corr:+.3f}")
print("\nInterpretation:")
print("  +1.0 = Perfect positive correlation (00 and 11)")
print("  -1.0 = Perfect negative correlation (01 and 10)")
print("   0.0 = No correlation")
print("="*70)

## CHSH Inequality: Proving Quantum Entanglement

**What is the CHSH inequality?**  
The CHSH (Clauser-Horne-Shimony-Holt) inequality is a quantitative test of quantum entanglement:

**Classical bound:** $|S| \leq 2$  
**Quantum bound:** $|S| \leq 2\sqrt{2} \approx 2.828$ (Tsirelson's bound)

Where $S = E(a,b) - E(a,b') + E(a',b) + E(a',b')$ and $E$ represents correlations at different measurement angles.

**What we'll do**  
Measure the Φ⁺ Bell state at different angles and compute the CHSH parameter S.

**Why**  
Violating the CHSH inequality ($S > 2$) proves that quantum correlations cannot be explained by any local hidden variable theory—a cornerstone result validating quantum mechanics.

**Expected result**  
$S \approx 2.828$, violating the classical bound and confirming quantum entanglement!

In [ ]:
def measure_bell_state_angles(angle_a, angle_b, shots=1000):
    """
    Measure Bell state with Alice at angle_a and Bob at angle_b.
    Returns correlation E(a,b).
    
    Standard CHSH protocol:
    - Measure along direction in X-Z plane rotated by angle from Z axis
    - Use RY rotation to align measurement axis with Z
    """
    # Create Phi+ Bell state
    qc = QuantumCircuit(2, 2)
    qc.h(0)
    qc.cx(0, 1)
    
    # Rotate measurement bases
    # RY(θ) rotates around Y axis by θ
    # To measure along axis at angle φ from Z in XZ plane, apply RY(-φ)
    qc.ry(-angle_a, 0)  # Alice's measurement angle
    qc.ry(-angle_b, 1)  # Bob's measurement angle
    
    qc.measure([0, 1], [0, 1])
    
    # Run simulation
    simulator = AerSimulator()
    job = simulator.run(qc, shots=shots)
    counts = job.result().get_counts()
    
    # Compute correlation
    correlation = compute_correlation_from_counts(counts)
    
    return correlation, counts

# Optimal angles for maximum CHSH violation
a = 0              # Alice's first angle
a_prime = np.pi/2  # Alice's second angle
b = np.pi/4        # Bob's first angle
b_prime = -np.pi/4 # Bob's second angle

print("Testing CHSH inequality with optimal measurement angles...\n")
print("="*70)

# Measure all four combinations
E_ab, counts_ab = measure_bell_state_angles(a, b, shots=5000)
E_ab_prime, counts_ab_prime = measure_bell_state_angles(a, b_prime, shots=5000)
E_a_prime_b, counts_a_prime_b = measure_bell_state_angles(a_prime, b, shots=5000)
E_a_prime_b_prime, counts_a_prime_b_prime = measure_bell_state_angles(a_prime, b_prime, shots=5000)

# Compute CHSH parameter S
# Correct CHSH formula: S = E(a,b) + E(a,b') + E(a',b) - E(a',b')
S = E_ab + E_ab_prime + E_a_prime_b - E_a_prime_b_prime

print(f"Measurement angles (in radians):")
print(f"  Alice:  a = {a:.3f}, a' = {a_prime:.3f}")
print(f"  Bob:    b = {b:.3f}, b' = {b_prime:.3f}")
print(f"\nCorrelations:")
print(f"  E(a, b)    = {E_ab:+.3f}")
print(f"  E(a, b')   = {E_ab_prime:+.3f}")
print(f"  E(a', b)   = {E_a_prime_b:+.3f}")
print(f"  E(a', b')  = {E_a_prime_b_prime:+.3f}")
print(f"\n" + "="*70)
print(f"CHSH PARAMETER: S = {S:.3f}")
print("="*70)
print(f"\nClassical bound:  |S| ≤ 2.000")
print(f"Quantum bound:    |S| ≤ {2*np.sqrt(2):.3f} (Tsirelson's bound)")
print(f"Measured value:   S = {S:.3f}")

if abs(S) > 2.0:
    print(f"\n🎉 VIOLATION! S > 2 confirms quantum entanglement!")
    print(f"   This proves quantum correlations cannot be explained classically.")
else:
    print(f"\n⚠️  No violation detected. Increase shots or check circuit.")

print("="*70)

In [ ]:
# Beautiful CHSH visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6), dpi=150)

# Left: Bar chart of correlations
corr_labels = ['E(a,b)', 'E(a,b\')', 'E(a\',b)', 'E(a\',b\')']
corr_values = [E_ab, E_ab_prime, E_a_prime_b, E_a_prime_b_prime]
colors_chsh = [COLORS['primary'], COLORS['secondary'], COLORS['accent'], COLORS['quantum']]

bars = ax1.bar(corr_labels, corr_values, color=colors_chsh, alpha=0.8, 
               edgecolor='black', linewidth=2)
ax1.axhline(y=0, color='black', linewidth=0.8)
ax1.set_ylabel('Correlation ⟨ZZ⟩', fontsize=13, fontweight='bold')
ax1.set_title('CHSH Correlations at Different Angles', fontsize=14, fontweight='bold')
ax1.set_ylim(-1.2, 1.2)
ax1.grid(axis='y', alpha=0.3)

# Add value labels
for bar in bars:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.3f}',
            ha='center', va='bottom' if height > 0 else 'top', 
            fontsize=11, fontweight='bold')

# Right: CHSH parameter comparison
bounds = ['Classical\nBound', 'Measured\nValue', 'Quantum\nBound']
values = [2.0, abs(S), 2*np.sqrt(2)]
colors_bounds = [COLORS['classical'], COLORS['success'], COLORS['quantum']]

bars2 = ax2.bar(bounds, values, color=colors_bounds, alpha=0.8,
                edgecolor='black', linewidth=2)
ax2.axhline(y=2.0, color='red', linewidth=2, linestyle='--', 
            label='Classical limit', alpha=0.7)
ax2.set_ylabel('|S| Value', fontsize=13, fontweight='bold')
ax2.set_title('CHSH Inequality: Quantum vs Classical', fontsize=14, fontweight='bold')
ax2.set_ylim(0, 3)
ax2.legend(fontsize=11)
ax2.grid(axis='y', alpha=0.3)

# Add value labels
for bar, val in zip(bars2, values):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
            f'{val:.3f}',
            ha='center', va='bottom', fontsize=12, fontweight='bold')

# Highlight violation
if abs(S) > 2.0:
    ax2.text(1, abs(S) + 0.15, '✓ Violation!', 
            ha='center', fontsize=12, color='green', fontweight='bold',
            bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.7))

plt.suptitle('CHSH Inequality Test: Proving Quantum Entanglement', 
             fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

print("\n⚛️  Bell's Theorem Confirmed: Quantum mechanics is fundamentally non-local!")

## Hardware Execution: Real Quantum Computer (Optional)

**What we'll do**  
Run Bell state creation and CHSH test on real quantum hardware (Compute Canada Monarch).

**Why**  
Entanglement is fragile—sensitive to noise, decoherence, and gate errors. Real hardware will show:
- Reduced CHSH violation (S closer to 2.0 than 2.828)
- Imperfect correlations due to noise
- Faster decoherence for entangled states

**Expected**  
CHSH parameter S ≈ 2.2-2.5 (still violating classical bound, but less than ideal).

In [ ]:
# Monarch hardware execution (placeholder)
from utils.monarch_config import MonarchConfig, print_hardware_info

print_hardware_info()

print("\n" + "⚠"*35)
print("HARDWARE EXECUTION PLACEHOLDER")
print("⚠"*35)
print("\nTo run Bell state experiments on Compute Canada Monarch:")
print("1. Configure credentials in utils/monarch_config.py")
print("2. Initialize MonarchConfig and connect to backend")
print("3. Transpile Bell state circuits for hardware")
print("4. Submit CHSH measurement jobs")
print("5. Compare simulator vs hardware CHSH values")
print("\nExpected hardware behavior:")
print("  • CHSH parameter S: 2.2-2.5 (reduced from ideal 2.828)")
print("  • Entanglement still visible but degraded")
print("  • Two-qubit gates are noisiest (affects CNOT in Bell state)")
print("  • Demonstrates robustness of quantum entanglement to noise")
print("⚠"*35)

## Summary: Quantum Entanglement Fundamentals

**What we learned**:

1. **Classical vs Quantum Correlations**
   - Classical: Limited to shared information, $C \leq 2$
   - Quantum: Entangled states violate classical bounds, $C \leq 2\sqrt{2}$

2. **Bell States: Four Maximally Entangled States**
   - $|\Phi^+\rangle$: Correlated (|00⟩ + |11⟩)
   - $|\Phi^-\rangle$: Correlated with phase
   - $|\Psi^+\rangle$: Anti-correlated (|01⟩ + |10⟩)
   - $|\Psi^-\rangle$: Anti-correlated with phase

3. **Perfect Correlations**
   - Measuring one qubit instantly determines the other
   - Works regardless of distance ("spooky action")
   - Cannot be used for faster-than-light communication

4. **CHSH Inequality: Bell's Theorem**
   - Classical bound: $|S| \leq 2$
   - Quantum violation: $S \approx 2.828$
   - Proves quantum mechanics is fundamentally non-local
   - No local hidden variable theory can reproduce quantum predictions

5. **Practical Implications**
   - **Quantum teleportation**: Transfer quantum states using entanglement
   - **Quantum cryptography**: Provably secure communication (BB84, E91)
   - **Quantum algorithms**: Entanglement enables speedups (Shor's, Grover's)
   - **Quantum error correction**: Protect information using entangled states

**Key insight**: Entanglement is not just a curious phenomenon—it's a computational resource. The correlations it provides cannot be achieved classically and are essential for quantum advantage.

**Next steps**:

In the next notebook, we'll use entanglement for **quantum teleportation**—transferring quantum states without physically moving particles. This demonstrates the most counter-intuitive application of entanglement!

---

*"I think I can safely say that nobody understands quantum mechanics."*  
— Richard Feynman

*"God does not play dice with the universe."*  
— Albert Einstein

*"Einstein, stop telling God what to do!"*  
— Niels Bohr

And quantum entanglement proved Bohr right. 🌌✨